<a href="https://colab.research.google.com/github/ashkwart/Ashley-Website/blob/main/Week13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config InlineBackend.figure_formats = ["retina"]

In [ ]:
!pip install -q folium geopandas

# Week 13: Neighborhoods
Based on ["An Extremely Detailed Map of New York City Neighborhoods"](https://www.nytimes.com/interactive/2023/upshot/extremely-detailed-nyc-neighborhood-map.html) (Buchanan et al. 2023)

In [ ]:
import json
from math import log

import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter
from shapely import MultiPolygon

In [ ]:
submissions_gdf = gpd.read_file("https://raw.githubusercontent.com/PUBPOL-2130/notebooks/refs/heads/main/data/nyt_neighborhood_submissions.geojson")

In [ ]:
submissions_gdf

In [ ]:
submissions_gdf.plot()

Each submission seems to have one neighborhood label.  These were chosen by the users together with their polygons.  There are 379 neighborhood labels.

In [ ]:
submissions_gdf["neighborhood"].value_counts()

It's not immediately obvious if users got to choose the neighborhood name as free text, or if they got a menu of options.  Let's see all the things called some kind of "village."

In [ ]:
submissions_gdf[submissions_gdf["neighborhood"].str.contains("illage")].value_counts("neighborhood")

The relative simplicity of this list strongly suggests that the NYT data folks presented people with this list of options rather than letting them enter their own free text.  Otherwise we'd expect lots of crazy misspellings and what-not!

Let's take a look at all the ones tagged as "Upper West Side"....

In [ ]:
submissions_map = folium.Map(
    [40.787, -73.9754],
    zoom_start=13,
    tiles="Cartodb Positron",  # use a less cluttered basemap
)
for _, row in submissions_gdf[submissions_gdf["neighborhood"] == "Upper West Side"].iterrows():
    folium.PolyLine([(y, x) for x, y in row.geometry.exterior.coords], weight=2, opacity=0.5).add_to(submissions_map)

In [ ]:
submissions_map

Amusingly, a few people don't know their East from West.... and one person is on the wrong land mass altogether.  These are the pleasures of user-submitted data!

The NYT folks did some processing, and in particular they worked out how often each census block got each tag.

In [ ]:
!curl -O https://raw.githubusercontent.com/PUBPOL-2130/notebooks/refs/heads/main/data/nyt_neighborhood_block_weights.json

In [ ]:
block_weights = json.load(open("nyt_neighborhood_block_weights.json"))

In [ ]:
list(block_weights.items())[:10]  # Census block GEOID -> neighborhood weights

## Neighborhood cores

In [ ]:
!curl -O https://www2.census.gov/geo/tiger/TIGER2024/TABBLOCK20/tl_2024_36_tabblock20.zip

In [ ]:
block_gdf = gpd.read_file("tl_2024_36_tabblock20.zip").set_index("GEOID20")

There are (at least) two ways of thinking about the level of disagreement.

Definition 1: of the submissions that include `block X`, what share labeled it as `neighborhood Y`? (in other words: what do New Yorkers call this block?)

Definition 2: of the submissions that are labeled with `neighborhood Y`, what share include `block X`? (in other words: among the people with an opinion on where that neighborhood is, do they include this block?)



So now we'll define a `core` function, which lets you choose a neighborhood and then picks all the blocks where at least `cutoff` share of submitters gave it that label.  (You can choose the cutoff!)

Note: This is based on the kind of consensus that's measured in Definition 1 above.

In [ ]:
def core(neighborhood, cutoff=0):  # def. (1)
    neighborhood_weights = {
        geoid: weights[neighborhood]
        for geoid, weights in block_weights.items()
        if neighborhood in weights and weights[neighborhood] >= cutoff
    }
    core_gdf = block_gdf.loc[neighborhood_weights.keys()]
    core_gdf["weight"] = neighborhood_weights
    return core_gdf.reset_index()

In [ ]:
core_map = folium.Map(
    [40.787, -73.9754],
    zoom_start=13,
    tiles="Cartodb Positron",  # use a less cluttered basemap
)
core_gdf = core("Upper West Side", cutoff=0.1)

folium.Choropleth(
    geo_data=core_gdf,
    data=core_gdf,
    columns=["GEOID20", "weight"],
    key_on="feature.properties.GEOID20",
    fill_color="OrRd",
).add_to(core_map)

Let's visualize (as a heatmap) Definition 1 of "Upper West Side."

In [ ]:
core_map

In [ ]:
block_neighborhood_counts = {
    geoid: len(weights)
    for geoid, weights in block_weights.items()
}

In [ ]:
plt.hist(block_neighborhood_counts.values(), bins=np.arange(1, 8)-0.5,rwidth=0.9)
plt.title("Neighborhood uncertainty: block counts by # of neighborhoods")
plt.xlabel("# of neighborhoods block is in")
plt.ylabel("Count")
plt.show()

For each block, we'll sum over its weight vector to get a measure of uncertainty about its neighborhood assignment.  We'll use the math concept called *Shannon entropy*, which was explained in class:  it is made by $H=\sum w_i \log w_i$ over the coordinates $i$.  So if a block is called "West Village" 100 times but "The Village" just once, its Shannon entropy is low, which means there's not significant disagreement.  But if it gets each label 50 times, it gets a higher score $H=1$.  And if it gets THREE labels each an equal number of times, it gets an even higher score $H=1.585$.  Basically you should think of $H$ as measuring the level of split-up-ness or controversy about the label.

In [ ]:
block_entropies = {
    geoid: -sum(v * log(v) for v in weights.values())
    for geoid, weights in block_weights.items()
}

In [ ]:
plt.hist(block_entropies.values(), bins=25)
plt.title("Neighborhood uncertainty: block counts by Shannon entropy")
plt.xlabel("Shannon entropy of neighborhood assignment")
plt.ylabel("Count")
plt.show()

In [ ]:
block_gdf["entropy"] = block_entropies
nyc_gdf = block_gdf.loc[block_entropies.keys()].reset_index()

In [ ]:
nyc_gdf.head(5)

In [ ]:
top_neighborhoods = submissions_gdf["neighborhood"].value_counts().head(100)
top_neighborhoods

In [ ]:
neighborhood_blocks = {neighborhood: [] for neighborhood in top_neighborhoods.keys()}
for geoid, weights in block_weights.items():
    for neighborhood, weight in weights.items():
        if neighborhood in neighborhood_blocks and weight > 0.3:  # increase this cutoff to tighten boundaries
            neighborhood_blocks[neighborhood].append(geoid)

In [ ]:
neighborhood_geos = {
    name: block_gdf.loc[geoids].dissolve().geometry
    for name, geoids in neighborhood_blocks.items()
}

And so we'll finish this notebook with a Folium map that plots the "core" neighborhoods (for the threshold you chose above).  And this should also have a checkbox in the upper right that lets you turn on and off a heatmap of the entropy score so you can see where there's the most label controversy!

In [ ]:
entropy_map = folium.Map(
    [40.65, -73.95],
    zoom_start=12,
    tiles="Cartodb Positron",
)
colors = ['#0099cd', '#ffca5d', '#00cd99', '#99cd00', '#cd0099', '#aa44ef', '#8dd3c7', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#bc80bd', '#ccebc5', '#ffed6f', '#ffffb3', '#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#b15928', '#64ffda', '#00B8D4', '#A1887F', '#76FF03', '#DCE775', '#B388FF', '#FF80AB', '#D81B60', '#26A69A', '#FFEA00', '#6200EA']

folium.Choropleth(
    geo_data=nyc_gdf,
    data=nyc_gdf,
    columns=["GEOID20", "entropy"],
    key_on="feature.properties.GEOID20",
    name="Neighborhood uncertainty (Shannon entropy)",
    fill_color="OrRd",
    line_opacity=0.2,
    fill_opacity=0.5,
    show=False,
).add_to(entropy_map)

for idx, (neighborhood, geos) in enumerate(neighborhood_geos.items()):
    if len(geos) > 0:
        for geo in geos:
            if isinstance(geo, MultiPolygon):
                for part in geo.geoms:
                    folium.Polygon(
                        [(y, x) for x, y in part.exterior.coords],
                        weight=6,
                        tooltip=neighborhood,
                        color=colors[idx % len(colors)],
                    ).add_to(entropy_map)
            else:
                folium.Polygon(
                    [(y, x) for x, y in geo.exterior.coords],
                    weight=6,
                    tooltip=neighborhood,
                    color=colors[idx % len(colors)],
                ).add_to(entropy_map)

folium.LayerControl(collapsed=False).add_to(entropy_map)

In [ ]:
entropy_map

#Homework 10 - extended due date Tuesday May 6, 1:25pm - recommended due date Friday May 2

**Warmup question:** How many blocks were given six different neighborhood labels by submitters?  Choose one of them and say where it is and what labels it received. See if you can find your chosen block on the interactive map, and upload a screenshot.

**Data product:** Get curious, formulate a question, make a good data product, and briefly explain how you made it.

**Reading question:** choose one.

IF YOU READ JACOBS: In this chapter, does she argue that ethnic neighborhoods are particiularly well suited to be powerful?  Explain how this fits into her overall argument about neighborhoods and policy power.

IF YOU READ THE [NYT GUIDE](https://www.nytimes.com/interactive/2023/10/29/upshot/new-york-neighborhood-guide.html): summarize the points made about Prospect Heights vs. Crown Heights.  Who has an incentive to shift what the blocks in the middle zone are called?

**Warm Up**

In [1]:
# Filter blocks with 6 neighborhood labels
six_label_blocks = [geoid for geoid, weights in block_weights.items() if len(weights) == 6]

# Count them
num_six_label_blocks = len(six_label_blocks)

print(f"There are {num_six_label_blocks} blocks with six different neighborhood labels.")

# Choose one block (the first one in the list)
chosen_block = six_label_blocks[700]

# Get its labels and weights
labels_and_weights = block_weights[chosen_block]

print(f"Chosen block: {chosen_block}")
print(f"Labels and weights: {labels_and_weights}")

# Get the block's geometry from the block_gdf
block_geometry = block_gdf.loc[chosen_block].geometry

# Get the block's centroid (center point)
centroid = block_geometry.centroid

print(f"Block centroid (latitude, longitude): {centroid.y}, {centroid.x}")

NameError: name 'block_weights' is not defined

**Data Product**

**Reading Response**